In [2]:
import timeit
import pandas as pd
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os

# Clean and tokenize text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords') # download stopwords corpus
nltk.download('punkt') # download punkt tokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Hannah/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/Hannah/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## XGBOOST

In [46]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


df = pd.read_csv('transformers_dataNew.csv')
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']

# In the first step we will split the data in training and remaining dataset

X_train, X_rem, y_train, y_rem = train_test_split(X , y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)

X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# create a Random Forest regressor object
# Define XGBoost model
XBBmodel = XGBRegressor(random_state = 42 )
XBBmodel.fit(X_train, y_train)

# make predictions on validation set
y_val_pred = XBBmodel.predict(X_valid)

# evaluate performance on validation set
val_mse = mean_squared_error(y_valid, y_val_pred)
print("Validation MSE:", val_mse)

# Define hyperparameter search space
params = {
    'max_depth': [3, 5, 7],
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.1, 0.01, 0.001]
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(model, param_grid=params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Print best hyperparameters
print('Best parameters:', grid_search.best_params_)

# Evaluate model on validation set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('Test MSE (tuned):', mse)

Validation MSE: 0.12276504895227135
Fitting 5 folds for each of 27 candidates, totalling 135 fits
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicit

## CATBOOST

In [27]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBRegressor

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

df = pd.read_csv('transformers_data.csv')

X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the CatBoost model
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3,
    loss_function='RMSE',
    verbose=False
)

# Train the model using the training set, and evaluate it using the testing set
model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    verbose=False
)

# Get the predictions on the testing set
y_pred = model.predict(X_test)

# Compute the MSE and R-squared
mse = mean_squared_error(y_test, y_pred)
rsq = r2_score(y_test, y_pred)

# Print the results
print("Mean Squared Error:", mse)
print("R-squared:", rsq)


## Random Forest

In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

df = pd.read_csv('transformers_dataNew.csv')
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X , y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# create a Random Forest regressor object
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# make predictions on validation set
y_val_pred = rf_model.predict(X_valid)

# evaluate performance on validation set
val_mse = mean_squared_error(y_valid, y_val_pred)
print("Validation MSE:", val_mse)

# create a GridSearchCV object and fit it to the data
rf_grid = GridSearchCV(rf_model, param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error')
rf_grid.fit(X_train, y_train)

# get the best model and print its hyperparameters
best_rf = rf_grid.best_estimator_
print("Best hyperparameters:", rf_grid.best_params_)

# evaluate the model on the test set
y_pred = best_rf.predict(X_test)
test_mse = mean_squared_error(y_pred, y_test)
print("Test MSE (tuned):", test_mse)

Validation MSE: 0.11828495625227002
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true |

## SVM

In [42]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error

df = pd.read_csv('transformers_dataNew.csv')
X = df.iloc[:, -768:-1]
#X = df['prep_content']
y = df['entropy']

# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(X , y, train_size=0.8)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

# initialize and train the SVM regressor
svm = LinearSVR(random_state=42)
svm.fit(X_train, y_train)

# make predictions on validation set
y_val_pred = svm.predict(X_valid)

# evaluate performance on validation set
val_mse = mean_squared_error(y_valid, y_val_pred)
print("Validation MSE:", val_mse)

# tune hyperparameters with GridSearchCV
params = {"C": [0.1, 1, 10], "epsilon": [0.1, 0.2, 0.3]}
svm_grid = GridSearchCV(svm, params, cv=5, scoring='neg_mean_squared_error')
svm_grid.fit(X_valid, y_valid)

# print best hyperparameters and corresponding performance on validation set
print("Best hyperparameters:", svm_grid.best_params_)
best_svm = svm_grid.best_estimator_
best_svm.fit(X_train, y_train)
y_pred = best_svm.predict(X_test)
test_mse = mean_squared_error(y_pred, y_test)
print("Test MSE (tuned):", test_mse)


Validation MSE: 0.13351362680688386
Best hyperparameters: {'C': 0.1, 'epsilon': 0.3}
Test MSE (tuned): 0.13003759970726186
